In [15]:
import readtop2000
import tqdm

In [4]:
URL_TOP2000_HISTORY = 'https://nl.wikipedia.org/wiki/Lijst_van_Radio_2-Top_2000%27s'

In [16]:
readtop2000

<module 'readtop2000' from 'C:\\Users\\Gebruiker\\Documents\\Willekeurige berekeningen\\Top2000\\readtop2000.py'>

## Base tables

In [5]:
def read():
    fulllist_reader = readtop2000.WikipediaTableExtractor(URL_TOP2000_HISTORY, [1], [0])
    df = fulllist_reader.extract_table_as_dataframe()
    
    cleaner = readtop2000.Top2000Cleaner(df)
    notering, song, songartist, artist = cleaner.clean()
    return notering, song, songartist, artist
notering, song, songartist, artist = read()

In [ ]:
artist[artist.groupby('Name')['Link'].transform('nunique').gt(1)]

## Detailed information for artists from inboxes
Drop Space Monkey because it does have an infobox, but it is about the song, not the artist

Drop Anita Garbo because the wikipedia redirects to a song, instead of to a page about her


In [ ]:
def download_infobox_details(links):
    result = []
    for link in tqdm.tqdm(links):
        info = readtop2000.InfoboxReader(link, allow_errors=True).read()
        result.append(info)
    return pd.concat(result)

links = artist.loc[~artist['Name'].isin(['Anita Garbo', 'Space Monkey']), 'Link']
artist_details = download_infobox_details(links)

 10%|████████                                                                       | 202/1967 [01:43<12:14,  2.40it/s]

In [ ]:
# The band members are much harder to handle because of all the functions they can have: it gives a many-to-many relation for members and bands
# So we ignore those. The same goes vice versa for the member pages in which it is discussed in what bands they were active
extra_artist_details = (ad[~ad['Header'].isin(['Leden', 'Oud-leden', 'Bezetting']) & ~ad['Header'].str.startswith('Actief')]  
                         .set_index(['OriginalLink', 'Variable'])['Value'].unstack())

In [ ]:
artist_full = artist.merge(extra_artist_details, right_index=True, left_on='Link', how='left')

## Save all tables

In [ ]:
tables = {'notering': notering,
          'song': song,
          'songartist': songartist,
          'artist_small': artist,
          'artist': artist_full,
          'artist_details': artist_details
         }
for name, table in tables.items():
    table.to_parquet(os.path.join('Data', f'{name}.parquet'))